# Setup

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image
%matplotlib inline
import networkx as nx

#from ipycytoscape import CytoscapeWidget
#import netgraph

#import ipycytoscape
#import ipywidgets as widgets
#%matplotlib widget

In [ ]:
#root_dir = '/mnt/d/biotools/workshop/'
root_dir='/root/jupyter_projects/ndm_plasmid_2021/'

# co-occurrence matrix

In [ ]:
#dir1 = 'ndm_plasmid/output/resfinder/all_ndm_complete_plasmid/'
dir1='input/'

In [ ]:
#header_line = ['File','Strain','Database','Resistance gene','Identity','Query / Template length', 'Contig','Position in contig','Predicted phenotype','Accession number']
df_resfinder = pd.read_csv(root_dir+dir1+'all_resfinder_c_p.tab',sep='\t',header=None)

In [ ]:
df1=df_resfinder.iloc[:][[1,13]]

In [ ]:
df1

In [ ]:
df1

In [ ]:
l2=[1 for i in range(3006)]

In [ ]:
df2=pd.concat([df1,pd.Series(l2)],axis=1)

In [ ]:
df2.columns=['ACC','AMR','num']

df2.AMR[df2.AMR.str.contains('NDM')]='blaNDM'

In [ ]:
df3=df2.pivot_table(index='ACC', columns='AMR', values='num',fill_value=0)

In [ ]:
df3

df3.to_csv(root_dir+dir1+'co_AMR.csv',float_format = '%.0f')

In [ ]:
df_asint = df3.astype(int)
coocc = df_asint.T.dot(df_asint)
np.fill_diagonal(coocc.values, 0)
coocc

coocc.to_csv(root_dir+dir1+'co_matrix.csv',float_format = '%.0f')

In [ ]:
coocc1=coocc.loc[(coocc != 0).any(axis=1), (coocc != 0).any(axis=0)]
coocc1

coocc1.to_csv(root_dir+dir1+'co_matrix_delall0.csv',float_format = '%.0f')

In [ ]:
coocc2=coocc.loc[(coocc > 19).any(axis=1), (coocc > 19).any(axis=0)]
coocc2

In [ ]:
i, j = np.triu_indices(len(coocc1), 1)

pd.DataFrame(pd.Series(
    coocc1.values[i, j], coocc1.index[i] +' '+ coocc1.index[j]
).sort_values(ascending=False))#.to_csv(root_dir+dir1+'co_sort1.txt',sep='\t',float_format = '%.0f',index=True,header=False)

In [ ]:
coocc.max(axis=0).max()

In [ ]:
sns.set(font_scale=5)

In [ ]:
plt.rc('font',family='sans-serif',size=50,style='normal')

COLOR = 'black'
plt.rcParams['text.color'] = COLOR
plt.rcParams['axes.labelcolor'] = COLOR

In [ ]:
cmap = sns.cubehelix_palette(start = 1.5, rot = 3, gamma=0.8, as_cmap = True)

In [ ]:
#fig_path='/mnt/c/Users/Yuing/Desktop/xiang2.pdf'
plt.figure(figsize=(80,60))
fig=sns.heatmap(coocc2, annot=True,cmap=cmap,linewidths=.02,fmt='d',cbar_kws={"shrink": .7})
#plt.show()
fig.set_xticklabels(fig.get_xmajorticklabels(), style='italic')
fig.set_yticklabels(fig.get_ymajorticklabels(), style='italic')
co_fig = fig.get_figure()
#co_fig.savefig(fig_path)

# co-occurrence network

In [ ]:
a = coocc2.stack()
a = a[a >= 1].rename_axis(('source', 'target')).reset_index(name='weight')
print(a)
b=a.sort_values('weight',ascending=False)
#b.to_csv('/root/jupyter_projects/ndm_plasmid_2021/output/file/'+'pairwise_co_AMR.txt')

In [ ]:
G = nx.from_pandas_edgelist(a,  edge_attr=True)
#nx.write_pajek(G, "/mnt/c/Users/Yuing/Desktop/test.net")
#print (nx.to_dict_of_dicts(G))

In [ ]:
nx.draw_spring(G)
plt.show()

cliq = nx.find_cliques(G)
list(cliq)

list(nx.make_max_clique_graph(G))

In [ ]:
list1=[40,50,60,70,80]
list2=['G'+str(i) for i in list1]
for i,Gi in zip(list1,list2):
    a1=[]
    for (u, v, wt) in G.edges.data('weight'):
        if wt >= i:
            a1.append((u, v, wt))
    locals()[Gi]=nx.Graph()
    locals()[Gi].add_weighted_edges_from(a1)

#nx.draw_networkx(G40)
#plt.show()
nx.write_gml(G50, "/mnt/c/Users/Yuing/Desktop/test50.gml")

nodes = nx.shortest_path(G,'blaNDM-1').keys()
s=G.subgraph(nodes)
nx.draw_spring(s)

pos
circular_layout：节点在一个圆环上均匀分布
random_layout：节点随机分布
shell_layout：节点在同心圆上分布
spring_layout： 用Fruchterman-Reingold算法排列节点（这个算法我不了解，样子类似多中心放射状）
spectral_layout：根据图的拉普拉斯特征向量排列节

fig = plt.figure(figsize=(8,8))

ax1 = plt.subplot2grid((6,6),(0,0),colspan=3,rowspan=4)#colspan宽度占格，rowspan高度占格
pos = nx.kamada_kawai_layout(G40)
#sizes = [30 + G40.degree(v) * 100 for v in G40.nodes()]  # 得到每个节点的度数
weights = nx.get_edge_attributes(G40,'weight').values()
nx.draw_networkx(G40,pos=pos,
with_labels=True, 
edge_color='crimson', width=2)
ax1.set_facecolor('white')
ax1.axis('off')

ax2 = plt.subplot2grid((6,6),(0,3),colspan=3,rowspan=4)
pos = nx.kamada_kawai_layout(G50)
sizes = [30 + G50.degree(v) * 100 for v in G50.nodes()]  # 得到每个节点的度数
weights = nx.get_edge_attributes(G50,'weight').values()
nx.draw_networkx(G50,pos=pos,
with_labels=True, 
edge_color='crimson', width=2)
ax2.set_facecolor('white')
ax2.axis('off')

ax5 =  plt.subplot2grid((6,6),(4,0),colspan=2,rowspan=2)
pos = nx.kamada_kawai_layout(G60)
sizes = [30 + G60.degree(v) * 100 for v in G60.nodes()]  # 得到每个节点的度数
weights = nx.get_edge_attributes(G60,'weight').values()
nx.draw_networkx(G60,pos=pos,
with_labels=True, 
edge_color='crimson', width=2)
ax5.set_facecolor('white')
ax5.axis('off')

ax3 = plt.subplot2grid((6,6),(4,2),colspan=2,rowspan=2)
pos = nx.kamada_kawai_layout(G70)
sizes = [30 + G70.degree(v) * 100 for v in G70.nodes()]  # 得到每个节点的度数
weights = nx.get_edge_attributes(G70,'weight').values()
nx.draw_networkx(G70,pos=pos,
with_labels=True, 
edge_color='crimson', width=2)
ax3.set_facecolor('white')
ax3.axis('off')


ax4 = plt.subplot2grid((6,6),(4,4),colspan=2,rowspan=2)
pos = nx.kamada_kawai_layout(G80)
sizes = [30 + G80.degree(v) * 100 for v in G80.nodes()]  # 得到每个节点的度数
weights = nx.get_edge_attributes(G80,'weight').values()
labels = {e: G80.edges[e]['weight'] for e in G80.edges}
nx.draw(G80, pos, with_labels=True,edge_color='crimson', width=2)
#nx.draw_networkx_edge_labels(G80,pos=pos)
ax4.set_facecolor('white')
ax4.axis('off')

#plt.tight_layout() 
#fig.set_facecolor('white')
plt.tight_layout()
#plt.savefig('/mnt/c/Users/Yuing/Desktop/conet2.pdf')
plt.show()


fig = plt.figure(figsize=(25,4))

ax1 = plt.subplot2grid((1,7),(0,0),colspan=2,rowspan=1)#colspan宽度占格，rowspan高度占格
pos = nx.kamada_kawai_layout(G40)
#sizes = [30 + G40.degree(v) * 100 for v in G40.nodes()]  # 得到每个节点的度数
weights = nx.get_edge_attributes(G40,'weight').values()
nx.draw_networkx(G40,pos=pos,
with_labels=True, 
edge_color='crimson', width=2)
ax1.set_facecolor('white')
ax1.axis('off')

ax2 = plt.subplot2grid((1,7),(0,2),colspan=2,rowspan=1)
pos = nx.kamada_kawai_layout(G50)
sizes = [30 + G50.degree(v) * 100 for v in G50.nodes()]  # 得到每个节点的度数
weights = nx.get_edge_attributes(G50,'weight').values()
nx.draw_networkx(G50,pos=pos,
with_labels=True, 
edge_color='crimson', width=2)
ax2.set_facecolor('white')
ax2.axis('off')

ax5 =  plt.subplot2grid((1,7),(0,4),colspan=1,rowspan=1)
pos = nx.kamada_kawai_layout(G60)
sizes = [30 + G60.degree(v) * 100 for v in G60.nodes()]  # 得到每个节点的度数
weights = nx.get_edge_attributes(G60,'weight').values()
nx.draw_networkx(G60,pos=pos,
with_labels=True, 
edge_color='crimson', width=2)
ax5.set_facecolor('white')
ax5.axis('off')

ax3 = plt.subplot2grid((1,7),(0,5),colspan=1,rowspan=1)
pos = nx.kamada_kawai_layout(G70)
sizes = [30 + G70.degree(v) * 100 for v in G70.nodes()]  # 得到每个节点的度数
weights = nx.get_edge_attributes(G70,'weight').values()
nx.draw_networkx(G70,pos=pos,
with_labels=True, 
edge_color='crimson', width=2)
ax3.set_facecolor('white')
ax3.axis('off')


ax4 = plt.subplot2grid((1,7),(0,6),colspan=1,rowspan=1)
pos = nx.kamada_kawai_layout(G80)
sizes = [30 + G80.degree(v) * 100 for v in G80.nodes()]  # 得到每个节点的度数
weights = nx.get_edge_attributes(G80,'weight').values()
labels = {e: G80.edges[e]['weight'] for e in G80.edges}
nx.draw(G80, pos, with_labels=True,edge_color='crimson', width=2)
#nx.draw_networkx_edge_labels(G80,pos=pos)
ax4.set_facecolor('white')
ax4.axis('off')

#plt.tight_layout() 
#fig.set_facecolor('white')
#plt.tight_layout()
#plt.savefig('/mnt/c/Users/Yuing/Desktop/conet2.pdf')
plt.show()


In [ ]:
gene_pa=df3.T

In [ ]:
ndm_clique=list(nx.find_cliques(G60))

In [ ]:
ndm_clique

In [ ]:
ndm1_include_clique = [clique for clique in ndm_clique if 'blaNDM-1' in clique]

In [ ]:
ndm5_include_clique = [clique for clique in ndm_clique if 'blaNDM-5' in clique]

In [ ]:
ndm1_include_clique

In [ ]:
ndm5_include_clique

In [ ]:
certain_clique=[['sul1','aadA2','dfrA12']]

In [ ]:
group_name=60
for item in ndm1_include_clique:
    I=[]
    df=gene_pa[gene_pa.index.isin(item)]
    #df.drop(['Gene'],axis=1,inplace=True)
    for index, col in df.iteritems():
        if col.sum() == df.shape[0]:
            I.append(index)
    #print(len(I))
    #print("{}\n".format(I))
    group_name+=1
    #with open('./'+str(group_name)+'.txt','w') as f:
        #f.write("{}\n".format("\n".join(str(i) for i in I)))

In [ ]:
#add multi-co information

In [ ]:
#dir2='ndm_plasmid/network/resfinder/'
dir2='input/resfinder/'

In [ ]:
#dir3='ndm_plasmid/output/'
dir3='input/'

In [ ]:
ndm_all_3=pd.read_csv(root_dir+dir3+'ndm_all_plsdb_merge_self_merge_mob.csv')

In [ ]:
ndm_species=ndm_all_3.loc[:][['ACC_NUCCORE','rep_type']]

In [ ]:
codes, uniques = pd.factorize(ndm_species.rep_type)

In [ ]:
sns.xkcd_palette()

In [ ]:
eight=sns.color_palette("Set2", 8).as_hex()
eight

In [ ]:
seven=sns.color_palette("Set1", 15).as_hex()[6:15]
seven

In [ ]:
sns_colors=eight+seven

In [ ]:
sns_colors

In [ ]:
plt.figure(figsize=(10,3))
fig=sns.color_palette(sns_colors)
fig

In [ ]:
colors_sp=pd.DataFrame(pd.Series(sns_colors,uniques),columns=['color'])

In [ ]:
colors_sp

In [ ]:
#custom
dir_customs=['40/1/41','40/1/42','40/1/43','40/1/44','40/1/45','40/1/46','40/1/47','40/5/41','50/1/52','50/1/53','50/5/51','60/61']          
pdf_names=['co_1_41_pie','co_1_42_pie','co_1_43_pie','co_1_44_pie','co_1_45_pie','co_1_46_pie','co_1_47_pie',
          'co_5_41_pie','co_1_52_pie','co_1_53_pie','co_5_51_pie','co_1_61_pie']

In [ ]:
import plotly.express as px
a = 2  # number of rows
b = 6  # number of columns
c = 1  # initialize plot counter
plt.figure(figsize=[30,10])

#for dir_custom,pdf_name in zip(dir_customs,pdf_names):
for dir_custom in dir_customs:

    co=pd.read_csv(root_dir+dir2+dir_custom+'.txt',sep='\t',header=None)
    co.columns=['ACC_NUCCORE']

    co_sp = pd.merge(co, ndm_species, how='left', on=['ACC_NUCCORE'])

    #co_sp['taxon_genus_name'].value_counts()
    unique=co_sp['rep_type'].value_counts().index
    #code, unique= pd.factorize(co_sp.taxon_genus_name)
    #print(unique)
    
    plt.subplot(a, b, c)
    patches, labels, pct_texts=plt.pie(co_sp['rep_type'].value_counts(),
       # kind='pie',
        autopct="%.1f%%",
        colors=colors_sp.loc[unique]['color'].values.tolist(),
        #title="",
        #explode=[0,0.1,0,0,0],                          
        labels=['']*(len(colors_sp.loc[unique]['color'].values.tolist())),
        textprops = {'fontsize':12,'color':"black",'family':'sans-serif'},
        #startangle=140,
        rotatelabels=True,
        #labeldistance=None,
        pctdistance=0.8,
        wedgeprops = {'linewidth': 0.01,'edgecolor': 'black'}
        #rotatelabels=True
        #frame=True
    )
    #rotate the texts
    for label, pct_text in zip(labels, pct_texts):
        pct_text.set_rotation(label.get_rotation())
    c=c+1
    #plt.tight_layout()
#plt.show()
    plt.legend(colors_sp.index.tolist(),ncol=2,loc='best',prop = {'size':11,'family':'sans-serif'})
plt.savefig(root_dir+dir2+'co_inc.pdf')

In [ ]:
!pip list

In [ ]:
!python3 --version